# Convert TFLite model to PyTorch

This uses the model **face_detection_front.tflite** from [MediaPipe](https://github.com/google/mediapipe/tree/master/mediapipe/models).

Prerequisites:

1) Clone the MediaPipe repo:

```
git clone https://github.com/google/mediapipe.git
```

2) Install **flatbuffers**:

```
git clone https://github.com/google/flatbuffers.git
cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release
make -j

cd flatbuffers/python
python setup.py install
```

3) Clone the TensorFlow repo. We only need this to get the FlatBuffers schema files (I guess you could just download [schema.fbs](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/schema/schema.fbs)).

```
git clone https://github.com/tensorflow/tensorflow.git
```

4) Convert the schema files to Python files using **flatc**:

```
./flatbuffers/flatc --python tensorflow/tensorflow/lite/schema/schema.fbs
```

Now we can use the Python FlatBuffer API to read the TFLite file!

In [ ]:
import os
import numpy as np
from collections import OrderedDict

## Get the weights from the TFLite file

Load the TFLite model using the FlatBuffers library:

In [ ]:
from tflite import Model

data = open("./mediapipe/mediapipe/models/face_detection_front.tflite", "rb").read()
model = Model.Model.GetRootAsModel(data, 0)

In [ ]:
subgraph = model.Subgraphs(0)
subgraph.Name()

In [ ]:
def get_shape(tensor):
    return [tensor.Shape(i) for i in range(tensor.ShapeLength())]

List all the tensors in the graph:

In [ ]:
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    print("%3d %30s %d %2d %s" % (i, tensor.Name(), tensor.Type(), tensor.Buffer(), 
                                  get_shape(subgraph.Tensors(i))))

Make a look-up table that lets us get the tensor index based on the tensor name:

In [ ]:
tensor_dict = {(subgraph.Tensors(i).Name().decode("utf8")): i 
               for i in range(subgraph.TensorsLength())}

Grab only the tensors that represent weights and biases.

In [ ]:
parameters = {}
for i in range(subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0:
        name = tensor.Name().decode("utf8")
        parameters[name] = tensor.Buffer()

len(parameters)

The buffers are simply arrays of bytes. As the docs say,

> The data_buffer itself is an opaque container, with the assumption that the
> target device is little-endian. In addition, all builtin operators assume
> the memory is ordered such that if `shape` is [4, 3, 2], then index
> [i, j, k] maps to `data_buffer[i*3*2 + j*2 + k]`.

For weights and biases, we need to interpret every 4 bytes as being as float. On my machine, the native byte ordering is already little-endian so we don't need to do anything special for that.

In [ ]:
def get_weights(tensor_name):
    i = tensor_dict[tensor_name]
    tensor = subgraph.Tensors(i)
    buffer = tensor.Buffer()
    shape = get_shape(tensor)
    assert(tensor.Type() == 0)  # FLOAT32
    
    W = model.Buffers(buffer).DataAsNumpy()
    W = W.view(dtype=np.float32)
    W = W.reshape(shape)
    return W

In [ ]:
W = get_weights("conv2d/Kernel")
b = get_weights("conv2d/Bias")
W.shape, b.shape

Now we can get the weights for all the layers and copy them into our PyTorch model.

## Convert the weights to PyTorch format

In [ ]:
import torch
from blazeface import BlazeFace

In [ ]:
net = BlazeFace()

In [ ]:
net

Make a lookup table that maps the layer names between the two models. We're going to assume here that the tensors will be in the same order in both models. If not, we should get an error because shapes don't match.

In [ ]:
probable_names = []
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0 and tensor.Type() == 0:
        probable_names.append(tensor.Name().decode("utf-8"))
        
probable_names[:5]

In [ ]:
convert = {}
i = 0
for name, params in net.state_dict().items():
    convert[name] = probable_names[i]
    i += 1

Copy the weights into the layers.

Note that the ordering of the weights is different between PyTorch and TFLite, so we need to transpose them.

Convolution weights:

    TFLite:  (out_channels, kernel_height, kernel_width, in_channels)
    PyTorch: (out_channels, in_channels, kernel_height, kernel_width)

Depthwise convolution weights:

    TFLite:  (1, kernel_height, kernel_width, channels)
    PyTorch: (channels, 1, kernel_height, kernel_width)

In [ ]:
new_state_dict = OrderedDict()

for dst, src in convert.items():
    W = get_weights(src)
    print(dst, src, W.shape, net.state_dict()[dst].shape)

    if W.ndim == 4:
        if W.shape[0] == 1:
            W = W.transpose((3, 0, 1, 2))  # depthwise conv
        else:
            W = W.transpose((0, 3, 1, 2))  # regular conv
    
    new_state_dict[dst] = torch.from_numpy(W)

In [ ]:
net.load_state_dict(new_state_dict, strict=True)

No errors? Then the conversion was successful!

## Save the checkpoint

In [ ]:
torch.save(net.state_dict(), "blazeface.pth")